## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

- https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python    
- http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer  
- https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction  
- https://pypi.org/project/rank-bm25/  

In [ ]:
from os import listdir, path
from pandas import *
import os, shutil, re
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from rank_bm25 import BM25Okapi
from nltk.tokenize import RegexpTokenizer
regex = "[\w+-]+|\([\s+\w+\d+-]+\)|\w+|\w"
tokenizex = RegexpTokenizer(regex)

corpus_global = []
base_path = '../02-filtrage/output/'
for file in listdir(base_path):
    if file.endswith('.csv'):
        with open(base_path + file, encoding = 'utf-8',) as f:
            corpus_global.append({'acteur': file[:-20], 'N_fr': 0, 'N_MeSH' : 0, 'N_Taxo' : 0}) 
            
N_termes = []
corpus_global

In [ ]:
for x in range(len(corpus_global)) :  
    acteur = corpus_global[x]['acteur']

    base_path = '../02-filtrage/output/'

    with open(base_path+acteur+'_candidate-terms.csv', encoding='utf-8') as f:
        csv = read_csv(f).drop(columns = ['Unnamed: 0'])

    corpus_global[x]['N_MeSH'] = len(csv[csv['isMeSHTerm'] == True])
    corpus_global[x]['N_Taxo'] = len(csv[csv['isTaxoTerm'] == True])

    # On va remplacer les apostrophes par des espaces ; sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: x.replace("'", ' '))
    vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]
    print(acteur, 'On a un vocabulaire de {} formes.'.format(len(vocabulaire)))
    N_termes += vocabulaire

    # Lire le corpus
    base_path = '../01-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur + '.csv')
        
    with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path, sep=',')
        text = data['text'].dropna().tolist()
        nb_docs = len(text)

    corpus_global[x]['N_fr'] = nb_docs
    print("On a un corpus de {} documents.".format(nb_docs))

    # Nettoyage
    corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    spaces = '\s+'

    corpus = [re.sub(spaces, ' ', t) for t in corpus]
    corpus = [t.replace("  ", " " ) for t in corpus]



    #On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 
    vectorizer = CountVectorizer(tokenizer = tokenizex.tokenize, vocabulary=vocabulaire, ngram_range=(1,10), token_pattern=regex)
    X = vectorizer.fit_transform(corpus)
    vectorizer.get_feature_names_out()

    features_names = vectorizer.get_feature_names_out()
    corpus_index = [corpus.index(n) for n in corpus]


    df = DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

    csv['TF (sklearn)'] = 0
    for t in vocabulaire:
        csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


    csv['DF (sklearn)'] = 0
    for t in vocabulaire:
        freqdoc = len(df[df[t] != 0])
        csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
    csv

    # Calcul TF-IDF
    # vocabulary = vocabulaire
    tfidf_vectorizer = TfidfVectorizer(tokenizer = tokenizex.tokenize, vocabulary=vocabulaire, ngram_range=(1,10), lowercase=False, token_pattern=regex)
    tfidf = tfidf_vectorizer.fit_transform(corpus)

    features_names = tfidf_vectorizer.get_feature_names_out()
    corpus_index = [corpus.index(n) for n in corpus]

    df = DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()
    terms_tfidf = {term: df[term].max() for term in df}
    zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}
    print('TF-IDF = 0',zeros)
    print('% TF-IDF = 0', (len(zeros) / len(vocabulaire) *100))
    
    terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])
    terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

    terms_weighted = terms_weighted.drop_duplicates(keep='first')

    terms_weighted = merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
    subset = ['Terme', 'Fréquence (TF)'],
    keep = 'first').reset_index(drop = True)


    # Calcul OKapi BM25
    tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")
    bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

    vocabulaire = csv["Terme"].dropna().tolist()
    tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

    features_names = [t for t in set(vocabulaire)]
    corpus_index = [corpus.index(n) for n in corpus]

    tab = [bm25.get_scores(query) for query in tokenized_queries]
    df = DataFrame(tab, index=features_names, columns=corpus_index).transpose()

    terms_okapi = {term: df[term].max() for term in df}
    tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
    tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

    tab = merge(terms_weighted, tab, on="Terme")
    tab.sort_values(["OkapiBM25"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    base_path = '../03-transformation/'
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
    tab.to_csv(file_path)